# [HydroSAR](https://github.com/HydroSAR/HydroSAR/tree/develop)

Set up the stack with help from HydroSAR [Notebooks](https://github.com/HydroSAR/HydroSAR/tree/develop)

In [1]:
import pandas as pd
import os
import subprocess
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

In [2]:
box_data = {
    'zipfile': ['S1A_IW_20250205T233956_DVP_RTC10_G_gduned_DD97.zip',
                'S1A_IW_20250217T233955_DVR_RTC10_G_gduned_0491.zip'],
    'link': ['https://ky.box.com/shared/static/2w2x6sj97x2trg0gv9az6k16o0v9zgbl.zip', 
             'https://ky.box.com/shared/static/w3zruxeczm2zpl9mepixbpp6nxszxzg1.zip']
    }

df = pd.DataFrame(data=box_data)

base_image = df.at[0,'zipfile']
flood_image = df.at[1, 'zipfile']
print(f'Base Image: {os.path.splitext(base_image)[0]}\nFlood Image: {os.path.splitext(flood_image)[0]}')

Base Image: S1A_IW_20250205T233956_DVP_RTC10_G_gduned_DD97
Flood Image: S1A_IW_20250217T233955_DVR_RTC10_G_gduned_0491


Function to download files from Box\* \*\*

\* At first I create a function to use the Box download link, but it was corrupting the file and not letting me unzip.  As a last resort, I'm using subprocess.

\*\* If all else fails, the download links can be used to directly download files and unzip locally

In [3]:
def box_download(df):
    out_folder = "data"
    download_zip = os.path.join(out_folder, 'file.zip')
    for row_index, row in df.iterrows():
        link = row['link']
        unzipped_folder = os.path.splitext(row['zipfile'])[0]  # get basename of zip file
        unzip_path = os.path.join(out_folder, unzipped_folder)

        cmd = f"curl -L -o {download_zip} {link} && unzip {download_zip} - d {unzip_path}"
        
        if not os.path.exists(unzip_path):
            subprocess.run(cmd, shell=True, check=True)
            # delete downloaded zip file.
            os.remove(download_zip)
        else:
            print("Extracted Files already exist")

I don't think using multiprocessing helps now that I'm using a subprocess, but it's not hurting it either.

Download could take several minutes (6 for me) depending upon bandwidth.

In [5]:
logical_cores = multiprocessing.cpu_count()
# determine number of threads to use for multiprocessing
num_workers = int(logical_cores * 0.75)  # rounds down in case not a whole number
print(f'Number of threads to use: {num_workers}')

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(box_download(df))

Number of threads to use: 7
Extracted Files already exist
Extracted Files already exist
